In [25]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os 

from scripts.vectology_functions import create_aaa_distances, create_pair_distances, embed_text, encode_traits
from nxontology import NXOntology


In [26]:
# get the EBI UKB data
#get ebi data
#url='https://raw.githubusercontent.com/EBISPOT/EFO-UKB-mappings/master/UK_Biobank_master_file.tsv'
#ebi_df = pd.read_csv(url,sep='\t')

ebi_df = pd.read_csv('data/UK_Biobank_master_file.tsv',sep='\t')

#drop some columns
ebi_df = ebi_df[['ZOOMA QUERY','MAPPED_TERM_LABEL','MAPPED_TERM_URI']]
ebi_df.rename(columns={'ZOOMA QUERY':'query'},inplace=True)
print(ebi_df.head())
print(ebi_df.shape)

#create new rows for multiple labels
#ebi_df = (
#        ebi_df.assign(label=ebi_df.MAPPED_TERM_LABEL.str.split("\|\|"))
#        .explode("label")
#        .reset_index(drop=True).drop('MAPPED_TERM_LABEL',axis=1)
#    )

#create new rows for multiple ids
ebi_df['MAPPED_TERM_URI']=ebi_df['MAPPED_TERM_URI'].str.replace('\|\|',',')
ebi_df['MAPPED_TERM_URI']=ebi_df['MAPPED_TERM_URI'].str.replace('\|',',')
ebi_df = (
        ebi_df.assign(id=ebi_df.MAPPED_TERM_URI.str.split(","))
        .explode("id")
        .reset_index(drop=True).drop('MAPPED_TERM_URI',axis=1)
    )

#clean up
ebi_df['id'] = ebi_df['id'].str.strip()

#drop cases where query and id are the same
ebi_df.drop_duplicates(subset=['query','id'],inplace=True)
print(ebi_df.shape)

#drop nan
ebi_df.dropna(inplace=True)
print(ebi_df.shape)
print(ebi_df.head())


                                      query       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1                              Gonarthrosis  osteoarthritis || knee   
2  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
3          Pain associated with micturition                 dysuria   
4                                Other mood           mood disorder   

            MAPPED_TERM_URI  
0  EFO_0004264, EFO_0009431  
1               EFO_0004616  
2               EFO_0003778  
3               EFO_0003901  
4               EFO_0004247  
(1565, 3)
(1613, 3)
(1600, 3)
                                      query       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1           Vascular disorders of intestine        vascular disease   
2                              Gonarthrosis  osteoarthritis || knee   
3  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
4       

In [27]:
%%time

#get EFO data
efo_data = 'data/efo-nodes.tsv'
efo_df=pd.read_csv(efo_data,sep='\t',names=['name','label','type'])
print(efo_df.head())
print(efo_df.shape)

                                        name                          label  \
0   http://www.orpha.net/ORDO/Orphanet_90342  Xeroderma pigmentosum variant   
1     http://www.orpha.net/ORDO/Orphanet_910          Xeroderma pigmentosum   
2  http://purl.obolibrary.org/obo/HP_0002140                Ischemic stroke   
3  http://purl.obolibrary.org/obo/HP_0002637              Cerebral ischemia   
4       http://www.ebi.ac.uk/efo/EFO_0008524           small cell carcinoma   

            type  
0  typed-literal  
1  typed-literal  
2  typed-literal  
3  typed-literal  
4  typed-literal  
(25390, 3)
CPU times: user 43.5 ms, sys: 7.09 ms, total: 50.6 ms
Wall time: 52 ms


In [28]:
#check all terms in EBI data set are here
efo_ids = list(efo_df['name'])
ebi_ids = list(ebi_df['id'])
missing=[]
matched = []
for i in ebi_ids:
    match = False
    for s in efo_ids:
        if i in s and match == False:
            matched.append(s)
            match = True
    if match == False:
        missing.append(i)
print(len(missing),missing)

# remove missing from ukb data
print(ebi_df.shape)
for i in missing:
    ebi_df = ebi_df.drop(ebi_df[ebi_df['id'].str.contains(i)].index)
ebi_df['full_id'] = matched
print(ebi_df.head())
print(ebi_df.shape)


21 ['HP_0001875', 'HP_0001888', 'EFO:0009626', 'NCIT_C27161', 'EFO:0009713', 'EFO:0009596', 'EFO_1000673', 'HP0011106', 'EFO_1000673', 'Orphanet:281097', 'EFO:0009640', 'EFO:0009642', 'EFO:0009642', 'EFO:0009643', 'EFO:0009642', 'EFO:0009642', 'EFO:0009639', 'EFO:0009638', 'EFO:0009636', 'HP_0100550', 'EFO:0009637']
(1600, 3)
                                      query       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1           Vascular disorders of intestine        vascular disease   
2                              Gonarthrosis  osteoarthritis || knee   
3  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
4          Pain associated with micturition                 dysuria   

            id                               full_id  
0  EFO_0004264  http://www.ebi.ac.uk/efo/EFO_0004264  
1  EFO_0009431  http://www.ebi.ac.uk/efo/EFO_0009431  
2  EFO_0004616  http://www.ebi.ac.uk/efo/EFO_0004616  
3  EFO_0003778  http://ww

In [29]:
# preprocess
process_text=[]
for k,g in ebi_df.groupby(np.arange(len(ebi_df))//20):
    params={'text_list':list(g['query'])}
    process_res = requests.post('http://vectology-api.mrcieu.ac.uk/preprocess',data=json.dumps(params))
    process_text.extend([d['result'].replace('unspecified','').replace('nec','') for d in process_res.json()])
print(len(process_text))
    
ebi_df.loc[:, 'processed'] = process_text
print(ebi_df.head())

1579
                                      query       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1           Vascular disorders of intestine        vascular disease   
2                              Gonarthrosis  osteoarthritis || knee   
3  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
4          Pain associated with micturition                 dysuria   

            id                               full_id  \
0  EFO_0004264  http://www.ebi.ac.uk/efo/EFO_0004264   
1  EFO_0009431  http://www.ebi.ac.uk/efo/EFO_0009431   
2  EFO_0004616  http://www.ebi.ac.uk/efo/EFO_0004616   
3  EFO_0003778  http://www.ebi.ac.uk/efo/EFO_0003778   
4  EFO_0003901  http://www.ebi.ac.uk/efo/EFO_0003901   

                                  processed  
0           vascular disorders of intestine  
1           vascular disorders of intestine  
2                              gonarthrosis  
3  psoriatic and enteropathic arthropathies  
4

In [30]:
%%time

f='data/ebi-ukb-vec.tsv.gz'
if os.path.exists(f):
    print('Already done')
    ebi_df = pd.read_csv(f,sep='\t')
    #issues with reading vectors from CSV 
    ebi_df['BioSentVec'] = ebi_df['BioSentVec'].apply(lambda x: [float(y) for y in x.replace('[','').replace(']','').split(',')])
else:
    ebi_df = encode_traits(trait_df=ebi_df,col='processed',name='BioSentVec',model='BioSentVec')
    ebi_df.to_csv(f,sep='\t',compression='gzip',index=False)
print(ebi_df.head())

1000 1579
1579 vectors created
                                      query       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1           Vascular disorders of intestine        vascular disease   
2                              Gonarthrosis  osteoarthritis || knee   
3  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
4          Pain associated with micturition                 dysuria   

            id                               full_id  \
0  EFO_0004264  http://www.ebi.ac.uk/efo/EFO_0004264   
1  EFO_0009431  http://www.ebi.ac.uk/efo/EFO_0009431   
2  EFO_0004616  http://www.ebi.ac.uk/efo/EFO_0004616   
3  EFO_0003778  http://www.ebi.ac.uk/efo/EFO_0003778   
4  EFO_0003901  http://www.ebi.ac.uk/efo/EFO_0003901   

                                  processed  \
0           vascular disorders of intestine   
1           vascular disorders of intestine   
2                              gonarthrosis   
3  psoriatic and 

In [31]:
%%time

f='data/efo-vec.tsv.gz'
if os.path.exists(f):
    print('Already done')
    efo_df = pd.read_csv(f,sep='\t')
    #issues with reading vectors from CSV 
    efo_df['BioSentVec'] = efo_df['BioSentVec'].apply(lambda x: [float(y) for y in x.replace('[','').replace(']','').split(',')])
else:
    efo_df = encode_traits(trait_df=efo_df,col='label',name='BioSentVec',model='BioSentVec')
    efo_df.to_csv(f,sep='\t',compression='gzip')
    
print(efo_df.head())


Already done
   Unnamed: 0                                       name  \
0           0   http://www.orpha.net/ORDO/Orphanet_90342   
1           1     http://www.orpha.net/ORDO/Orphanet_910   
2           2  http://purl.obolibrary.org/obo/HP_0002140   
3           3  http://purl.obolibrary.org/obo/HP_0002637   
4           4       http://www.ebi.ac.uk/efo/EFO_0008524   

                           label           type  \
0  Xeroderma pigmentosum variant  typed-literal   
1          Xeroderma pigmentosum  typed-literal   
2                Ischemic stroke  typed-literal   
3              Cerebral ischemia  typed-literal   
4           small cell carcinoma  typed-literal   

                                          BioSentVec  
0  [-0.045969586819410324, 0.5196870565414429, -0...  
1  [0.01842655800282955, 0.7100013494491577, -0.7...  
2  [0.5974032282829285, -0.6174705028533936, -0.1...  
3  [0.46593865752220154, -0.6334501504898071, 0.0...  
4  [0.4248107373714447, 0.033654093742370605

In [73]:
#create nxontology network of EFO relationships
def create_efo_nxo() -> NXOntology:
    nxo = NXOntology()
    
    edges = []
    efo_data='data/efo_data.txt.gz'
    efo_df=pd.read_json(efo_data)
    for i,row in efo_df.iterrows():
        child = row['child']['value']
        parent = row['parent']['value']
        edges.append((parent,child))
    print(edges[0:10])
    nxo.graph.add_edges_from(edges)
    return nxo

efo_nx = create_efo_nxo()
efo_nx.freeze()

[('http://purl.obolibrary.org/obo/HP_0000093', 'http://www.ebi.ac.uk/efo/EFO_0004285'), ('http://purl.obolibrary.org/obo/HP_0000545', 'http://www.ebi.ac.uk/efo/EFO_0004207'), ('http://purl.obolibrary.org/obo/HP_0000726', 'http://www.ebi.ac.uk/efo/EFO_0002608'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0000132'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0001873'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0001915'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0100727'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://www.ebi.ac.uk/efo/EFO_0004233'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://www.ebi.ac.uk/efo/EFO_0004272'), ('http://purl.obolibrary.org/obo/HP_0002017', 'http://purl.obolibrary.org/obo/HP_0002018')]


In [32]:
%%time

#ebi_df['BioSentVec']
v1 = list(ebi_df['BioSentVec'])

v2 = list(efo_df['BioSentVec'])


CPU times: user 191 ms, sys: 116 ms, total: 307 ms
Wall time: 308 ms


In [33]:
%%time
# cosine of all against all
dd = create_pair_distances(v1,v2)

Creating distances...
1579 25390
1579
CPU times: user 30 s, sys: 537 ms, total: 30.5 s
Wall time: 30.7 s


In [78]:
for i,irows in ebi_df.head(n=5).iterrows():
    print('\nEBI:',i,irows[['query','full_id']].values)
    min_dis = np.nanargmin(dd[i])
    print('EFO:',efo_df.iloc[min_dis][['name','label']].values)
    try:
        similarity = efo_nx.similarity(irows['full_id'],efo_df.iloc[min_dis]['name'])
        print(similarity.results())
    except:
        print('something wrong')



EBI: 0 ['Vascular disorders of intestine' 'http://www.ebi.ac.uk/efo/EFO_0004264']
EFO: ['http://purl.obolibrary.org/obo/UBERON_0000160' 'intestine']
{'node_0': 'http://www.ebi.ac.uk/efo/EFO_0004264', 'node_1': 'http://purl.obolibrary.org/obo/UBERON_0000160', 'node_0_subsumes_1': False, 'node_1_subsumes_0': False, 'n_common_ancestors': 1, 'n_union_ancestors': 11, 'batet': 0.09090909090909091, 'batet_log': 0.03974743221087251, 'ic_metric': 'intrinsic_ic_sanchez', 'mica': 'http://www.ebi.ac.uk/efo/EFO_0000001', 'resnik': 0.08349779164953475, 'resnik_scaled': 0.008505661843857603, 'lin': 0.009549764063794623, 'jiang': 0.05458549563831071, 'jiang_seco': 0.11783843833573482}

EBI: 1 ['Vascular disorders of intestine' 'http://www.ebi.ac.uk/efo/EFO_0009431']
EFO: ['http://purl.obolibrary.org/obo/UBERON_0000160' 'intestine']
something wrong

EBI: 2 ['Gonarthrosis' 'http://www.ebi.ac.uk/efo/EFO_0004616']
EFO: ['http://www.ebi.ac.uk/efo/EFO_0004616' 'osteoarthritis, knee']
{'node_0': 'http://www

In [64]:
def create_efo_nxo() -> NXOntology:
    nxo = NXOntology()
    
    edges = []
    efo_data='data/efo_data.txt.gz'
    efo_df=pd.read_json(efo_data)
    for i,row in efo_df.iterrows():
        child = row['child']['value']
        parent = row['parent']['value']
        edges.append((parent,child))
    print(edges[0:10])
    nxo.graph.add_edges_from(edges)
    return nxo

efo_nx = create_efo_nxo()
efo_nx.freeze()

[('http://purl.obolibrary.org/obo/HP_0000093', 'http://www.ebi.ac.uk/efo/EFO_0004285'), ('http://purl.obolibrary.org/obo/HP_0000545', 'http://www.ebi.ac.uk/efo/EFO_0004207'), ('http://purl.obolibrary.org/obo/HP_0000726', 'http://www.ebi.ac.uk/efo/EFO_0002608'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0000132'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0001873'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0001915'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://purl.obolibrary.org/obo/HP_0100727'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://www.ebi.ac.uk/efo/EFO_0004233'), ('http://purl.obolibrary.org/obo/HP_0001871', 'http://www.ebi.ac.uk/efo/EFO_0004272'), ('http://purl.obolibrary.org/obo/HP_0002017', 'http://purl.obolibrary.org/obo/HP_0002018')]


In [71]:
similarity = efo_nx.similarity("http://www.ebi.ac.uk/efo/EFO_0003901","http://purl.obolibrary.org/obo/GO_0060073")
similarity.results()

{'node_0': 'http://www.ebi.ac.uk/efo/EFO_0003901',
 'node_1': 'http://purl.obolibrary.org/obo/GO_0060073',
 'node_0_subsumes_1': False,
 'node_1_subsumes_0': False,
 'n_common_ancestors': 1,
 'n_union_ancestors': 13,
 'batet': 0.07692307692307693,
 'batet_log': 0.031206350113965813,
 'ic_metric': 'intrinsic_ic_sanchez',
 'mica': 'http://www.ebi.ac.uk/efo/EFO_0000001',
 'resnik': 0.08349779164953475,
 'resnik_scaled': 0.008505661843857603,
 'lin': 0.00862512432988957,
 'jiang': 0.0495183545781566,
 'jiang_seco': 0.022356185205935852}

In [72]:
similarity = efo_nx.similarity("http://purl.obolibrary.org/obo/GO_0060073","http://www.ebi.ac.uk/efo/EFO_0003901")
similarity.results()

{'node_0': 'http://purl.obolibrary.org/obo/GO_0060073',
 'node_1': 'http://www.ebi.ac.uk/efo/EFO_0003901',
 'node_0_subsumes_1': False,
 'node_1_subsumes_0': False,
 'n_common_ancestors': 1,
 'n_union_ancestors': 13,
 'batet': 0.07692307692307693,
 'batet_log': 0.031206350113965813,
 'ic_metric': 'intrinsic_ic_sanchez',
 'mica': 'http://www.ebi.ac.uk/efo/EFO_0000001',
 'resnik': 0.08349779164953475,
 'resnik_scaled': 0.008505661843857603,
 'lin': 0.00862512432988957,
 'jiang': 0.0495183545781566,
 'jiang_seco': 0.022356185205935852}